**This notebook is an exercise in the [Advanced SQL](https://www.kaggle.com/learn/advanced-sql) course.  You can reference the tutorial at [this link](https://www.kaggle.com/alexisbcook/analytic-functions).**

---


# Introduction

Here, you'll use window functions to answer questions about the [Chicago Taxi Trips](https://www.kaggle.com/chicago/chicago-taxi-trips-bq) dataset.

Before you get started, run the code cell below to set everything up.

In [2]:
# Get most recent checking code
!pip install -U -t /kaggle/working/ git+https://github.com/Kaggle/learntools.git
# Set up feedback system
from learntools.core import binder
binder.bind(globals())
from learntools.sql_advanced.ex2 import *
print("Setup Complete")

  Cloning https://github.com/Kaggle/learntools.git to /tmp/pip-req-build-bcebcnxc
  Running command git clone --filter=blob:none --quiet https://github.com/Kaggle/learntools.git /tmp/pip-req-build-bcebcnxc
  fatal: unable to access 'https://github.com/Kaggle/learntools.git/': Could not resolve host: github.com
  error: subprocess-exited-with-error
  
  × git clone --filter=blob:none --quiet https://github.com/Kaggle/learntools.git /tmp/pip-req-build-bcebcnxc did not run successfully.
  │ exit code: 128
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: subprocess-exited-with-error

× git clone --filter=blob:none --quiet https://github.com/Kaggle/learntools.git /tmp/pip-req-build-bcebcnxc did not run successfully.
│ exit code: 128
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.
Setup Complete


The following code cell fetches the `taxi_trips` table from the `chicago_taxi_trips` dataset. We also preview the first five rows of the table.  You'll use the table to answer the questions below.

In [3]:
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "chicago_taxi_trips" dataset
dataset_ref = client.dataset("chicago_taxi_trips", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

# Construct a reference to the "taxi_trips" table
table_ref = dataset_ref.table("taxi_trips")

# API request - fetch the table
table = client.get_table(table_ref)

# Preview the first five lines of the table
client.list_rows(table, max_results=5).to_dataframe()

Using Kaggle's public dataset BigQuery integration.


,unique_key,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,...,extras,trip_total,payment_type,company,pickup_latitude,pickup_longitude,pickup_location,dropoff_latitude,dropoff_longitude,dropoff_location
0,5f5fb47c42411bd0d86c13435af2c5dfbc08c186,06e95e5723605c6969624e7c1493e6ccf7e08a850e805a...,2016-05-11 12:15:00+00:00,2016-05-11 13:15:00+00:00,3603,40.2,NaN,NaN,56,NaN,...,0.0,0.01,Cash,303 Taxi,41.792592,-87.769615,POINT (-87.7696154528 41.7925923603),NaN,NaN,None
1,7f9c936c4920901c8267b926d9e6c5e369d88f33,d2c2d4128d6597a3b6db0bd0efa96e9914be3994f861bb...,2016-07-26 10:45:00+00:00,2016-07-26 10:45:00+00:00,79,0.2,1.703184e+10,1.703184e+10,32,32.0,...,0.0,4.75,Credit Card,Flash Cab,41.880994,-87.632746,POINT (-87.6327464887 41.8809944707),41.880994,-87.632746,POINT (-87.6327464887 41.8809944707)
2,557cfc87719c0d54a0036c88069ae78efc0ea24b,1941206105203a6a747f261477b39bd3e4f5876ff31710...,2016-07-26 09:30:00+00:00,2016-07-26 09:45:00+00:00,630,3.3,NaN,NaN,28,23.0,...,1.0,12.50,Cash,Flash Cab,41.874005,-87.663518,POINT (-87.6635175498 41.874005383),41.900070,-87.720918,POINT (-87.7209182385 41.9000696026)
3,0fb433d93583294343035881bf3bbb3a3b009f84,aeae74b676d686b382b5a5ad197aa0b7dded703bce5bf3...,2016-07-28 19:00:00+00:00,2016-07-28 19:30:00+00:00,1909,5.3,1.703132e+10,1.703106e+10,32,6.0,...,0.0,19.25,Cash,Flash Cab,41.884987,-87.620993,POINT (-87.6209929134 41.8849871918),41.942692,-87.651771,POINT (-87.6517705068 41.9426918444)
4,7329101487bc6d4fdfcfb58026870fe869cac184,432e78c6d610d1193b4fce3766710c5f7302437cf98b24...,2016-07-15 16:00:00+00:00,2016-07-15 16:15:00+00:00,66,0.1,1.703198e+10,1.703198e+10,76,76.0,...,0.0,0.01,Cash,303 Taxi,41.979071,-87.903040,POINT (-87.9030396611 41.9790708201),41.979071,-87.903040,POINT (-87.9030396611 41.9790708201)


# Exercises

### 1) How can you predict the demand for taxis?

Say you work for a taxi company, and you're interested in predicting the demand for taxis.  Towards this goal, you'd like to create a plot that shows a rolling average of the daily number of taxi trips.  Amend the (partial) query below to return a DataFrame with two columns:
- `trip_date` - contains one entry for each date from January 1, 2016, to March 31, 2016.
- `avg_num_trips` - shows the average number of daily trips, calculated over a window including the value for the current date, along with the values for the preceding 3 days and the following 3 days, as long as the days fit within the three-month time frame.  For instance, when calculating the value in this column for January 3, 2016, the window will include the number of trips for the preceding 2 days, the current date, and the following 3 days.

This query is partially completed for you, and you need only write the part that calculates the `avg_num_trips` column.  Note that this query uses a common table expression (CTE); if you need to review how to use CTEs, you're encouraged to check out [this tutorial](https://www.kaggle.com/dansbecker/as-with) in the [Intro to SQL](https://www.kaggle.com/learn/intro-to-sql) course.

In [4]:
# Fill in the blank below
avg_num_trips_query = """
                      WITH trips_by_day AS
                      (
                      SELECT DATE(trip_start_timestamp) AS trip_date,
                          COUNT(*) as num_trips
                      FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                      WHERE trip_start_timestamp > '2016-01-01' AND trip_start_timestamp < '2016-04-01'
                      GROUP BY trip_date
                      )
                      SELECT trip_date,
                          AVG(num_trips) 
                          OVER (
                               ORDER BY trip_date
                               ROWS BETWEEN 3 PRECEDING AND 3 FOLLOWING
                               ) AS avg_num_trips
                      FROM trips_by_day
                      """

# Check your answer
q_1.check()

,trip_date,avg_num_trips
0,2016-03-22,90167.000000
1,2016-02-16,90901.142857
2,2016-03-06,92647.000000
3,2016-03-12,99863.428571
4,2016-03-15,103229.142857


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [ ]:
# Lines below will give you a hint or solution code
# q_1.hint()
# q_1.solution()

### 2) Can you separate and order trips by community area?

The query below returns a DataFrame with three columns from the table: `pickup_community_area`, `trip_start_timestamp`, and `trip_end_timestamp`.  

Amend the query to return an additional column called `trip_number` which shows the order in which the trips were taken from their respective community areas.  So, the first trip of the day originating from community area 1 should receive a value of 1; the second trip of the day from the same area should receive a value of 2.  Likewise, the first trip of the day from community area 2 should receive a value of 1, and so on.

Note that there are many numbering functions that can be used to solve this problem (depending on how you want to deal with trips that started at the same time from the same community area); to answer this question, please use the **RANK()** function.

In [5]:
# Amend the query below
trip_number_query = """
                    SELECT pickup_community_area,
                        trip_start_timestamp,
                        trip_end_timestamp,
                        RANK()
                            OVER (
                                  PARTITION BY pickup_community_area
                                  ORDER BY trip_start_timestamp
                                 ) AS trip_number
                    FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                    WHERE DATE(trip_start_timestamp) = '2013-10-03' 
                    """

trip_number_result = client.query(trip_number_query).result().to_dataframe()

# Check your answer
q_2.check()

,pickup_community_area,trip_start_timestamp,trip_end_timestamp,trip_number
0,36.0,2013-10-03 03:30:00+00:00,2013-10-03 03:45:00+00:00,1
1,36.0,2013-10-03 06:45:00+00:00,2013-10-03 06:45:00+00:00,2
2,36.0,2013-10-03 09:15:00+00:00,2013-10-03 09:30:00+00:00,3
3,36.0,2013-10-03 10:30:00+00:00,2013-10-03 10:30:00+00:00,4
4,36.0,2013-10-03 13:15:00+00:00,2013-10-03 13:30:00+00:00,5


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [ ]:
# Lines below will give you a hint or solution code
# q_2.hint()
# q_2.solution()

### 3) How much time elapses between trips?

The (partial) query in the code cell below shows, for each trip in the selected time frame, the corresponding `taxi_id`, `trip_start_timestamp`, and `trip_end_timestamp`. 

Your task in this exercise is to edit the query to include an additional `prev_break` column that shows the length of the break (in minutes) that the driver had before each trip started (this corresponds to the time between `trip_start_timestamp` of the current trip and `trip_end_timestamp` of the previous trip).  Partition the calculation by `taxi_id`, and order the results within each partition by `trip_start_timestamp`.

Some sample results are shown below, where all rows correspond to the same driver (or `taxi_id`).  Take the time now to make sure that the values in the `prev_break` column make sense to you!

![first_commands](https://storage.googleapis.com/kaggle-media/learn/images/qjvQzg8.png)

Note that the first trip of the day for each driver should have a value of **NaN** (not a number) in the `prev_break` column.

In [6]:
# Fill in the blanks below
break_time_query = """
                   SELECT taxi_id,
                       trip_start_timestamp,
                       trip_end_timestamp,
                       TIMESTAMP_DIFF(
                           trip_start_timestamp, 
                           LAG(trip_end_timestamp, 1) OVER (PARTITION BY taxi_id ORDER BY trip_start_timestamp), 
                           MINUTE) as prev_break
                   FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                   WHERE DATE(trip_start_timestamp) = '2013-10-03' 
                   """

break_time_result = client.query(break_time_query).result().to_dataframe()

# Check your answer
q_3.check()

,taxi_id,trip_start_timestamp,trip_end_timestamp,prev_break
0,01fbd58ba9edb672d7f26b8c367871e49809b05ddc6f31...,2013-10-03 16:30:00+00:00,2013-10-03 16:30:00+00:00,180.0
1,02b8e2d9557a991c66ea3872d69cc1a8e0472d72a9b0bc...,2013-10-03 18:15:00+00:00,2013-10-03 18:45:00+00:00,300.0
2,18c20740c9cbe78a07c7bef21fd4dd4035fa01fb10f186...,2013-10-03 20:30:00+00:00,2013-10-03 20:30:00+00:00,270.0
3,2155f293be9d55aca9fe24325dfeb7d8b821ca8c29adc9...,2013-10-03 11:00:00+00:00,2013-10-03 11:00:00+00:00,210.0
4,372ab351f35537e6d7a4ae2b3cd053341e83908e13d436...,2013-10-03 06:15:00+00:00,2013-10-03 06:45:00+00:00,315.0


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [ ]:
# Lines below will give you a hint or solution code
# q_3.hint()
# q_3.solution()

# Keep going

Move on to learn how to query **[nested and repeated data](https://www.kaggle.com/alexisbcook/nested-and-repeated-data)**.

---




*Have questions or comments? Visit the [course discussion forum](https://www.kaggle.com/learn/advanced-sql/discussion) to chat with other learners.*